In [2]:
import pandas as pd
import httpx
from string import Template
import regex as re 

In [4]:
path = 'data/positions_salaries.csv'
test = pd.read_csv(path)
test

,job_title,job_description,salary_string
0,Spa Therapist,· High school qualification with minimum 2 yea...,"QAR 4,000 - QAR 5,000 a month"
1,Registered Triage Nurse NHS (Relocation Package),Triage Nurse (Registered Nurse) – Unique Oppor...,"QAR 148,490 - QAR 178,808 a year"
2,Sales & Marketing specialist - Food & Beverage,● Minimum Experience of 3 Years in F&B Devisio...,"QAR 5,000 - QAR 6,000 a month"
3,Food Service Crew (Female),LOCAL HIRE (Must be in Doha-Qatar)\nPreferably...,"QAR 2,500 a month"
4,Digital Print Operator,We are hiring - Digital Print Operator\nQAR 35...,"QAR 3,400 - QAR 3,500 a month"
...,...,...,...
995,Road Chargehand (TC23002),Emirates General Services Enterprises (_EGSE_)...,"AED3,000 a month"
996,Internal Auditor,Perform and control the full audit cycle inclu...,"AED7,000 - AED7,500 a month"
997,Tele sales Executive (Female),We are looking for an enthusiastic Tele sales ...,"AED2,500 - AED3,500 a month"
998,Commis Chef,We are looking to hire a hard-working and effi...,"AED2,500 - AED3,500 a month"


In [71]:
path = 'data/job_descriptions_modified.csv' 
df = pd.read_csv(path)
df

,job_title,job_description,salary_string,currency,salary_lower,salary_upper,salary_avg,Predicted Category,Predicted Level
0,Spa Therapist,· High school qualification with minimum 2 yea...,"QAR 4,000 - QAR 5,000 a month",QAR,4000.000000,5000.000000,4500.000000,IT,assistant
1,Registered Triage Nurse NHS (Relocation Package),Triage Nurse (Registered Nurse) – Unique Oppor...,"QAR 148,490 - QAR 178,808 a year",QAR,12374.166667,14900.666667,13637.416667,nurse,senior
2,Sales & Marketing specialist - Food & Beverage,● Minimum Experience of 3 Years in F&B Devisio...,"QAR 5,000 - QAR 6,000 a month",QAR,5000.000000,6000.000000,5500.000000,food and beverage,senior
3,Food Service Crew (Female),LOCAL HIRE (Must be in Doha-Qatar)\nPreferably...,"QAR 2,500 a month",QAR,2500.000000,2500.000000,2500.000000,food and beverage,assistant
4,Digital Print Operator,We are hiring - Digital Print Operator\nQAR 35...,"QAR 3,400 - QAR 3,500 a month",QAR,3400.000000,3500.000000,3450.000000,IT,assistant
...,...,...,...,...,...,...,...,...,...
995,Road Chargehand (TC23002),Emirates General Services Enterprises (_EGSE_)...,"AED3,000 a month",AED,3000.000000,3000.000000,3000.000000,HR,assistant
996,Internal Auditor,Perform and control the full audit cycle inclu...,"AED7,000 - AED7,500 a month",AED,7000.000000,7500.000000,7250.000000,HR,intern
997,Tele sales Executive (Female),We are looking for an enthusiastic Tele sales ...,"AED2,500 - AED3,500 a month",AED,2500.000000,3500.000000,3000.000000,sales,executive
998,Commis Chef,We are looking to hire a hard-working and effi...,"AED2,500 - AED3,500 a month",AED,2500.000000,3500.000000,3000.000000,food and beverage,executive


In [69]:
OLLAMA_ENDPOINT = "http://localhost:11434/api/generate"
OLLAMA_CONFIG = {
    "model": "mistral:7b-instruct-v0.2-q4_K_S",
    "keep_alive": "5m",
    "stream": False,
}

PROMPT_TEMPLATE = Template(
    """Extract the following details from the job description below, Ensure each response is concise, accurate, and formatted as specified::
    - **Job Title**: Provide the specific job title mentioned.
    - **Experience**: Extract the required experience as a single number representing years. If it's less than 1 year, use "0" (without text).
    - **Expected Salary Range**: Provide the numerical salary range in "x-y" format (without currency symbols).
    - **Expected Salary Currency**: Extract one main currency from the following options: USD, EUR, GBP, INR, JPY, AED, QAR, SAR, KD, JD.
    - **Job Type**: Extract the job type (choose from: Full-time, Part-time, Contract, Freelance, Internship, Temporary).
    - **Location**: Provide the country name only.
    - **Education Level**: Extract the education level choose one(choose from: School, High School, Diplomate, College, Bachelor, Master, PhD, Other).
    - **Major**: Extract one major (choose from: Business, Health, Social sciences and history, Engineering, Biological and biomedical sciences, Psychology, Computer and information sciences, Visual and performing arts, Communication and journalism, Education).
    - **Level**: Extract the level of the role (choose from: Senior,Mid ,Normal, Junior, Manager, Executive, Assistant, Intern, Lead, Director).
    - **Job Function**: Extract one job function (choose from: Developer, Data Scientist, Marketing, Health, Sales, Finance, Customer Service, IT, Teacher, HR, Food and Beverage, Real Estate, Admin, Construction, other).
    - **Industry**: Provide the industry type choose one(Technology, Healthcare, Education, Retail, Manufacturing, other).


    Job Description: $text

    Respond in JSON format with the keys: "Job Title", "Experience", "Expected Salary Range","Expected Salary Currency", "Job Type", "Location", "Education Level", "Major", "Level", "Job Function",  "Industry".
    As 
    - "Job Title"
    - "Experience"
    - "Salary Range"
    - "Salary Currency"
    - "Job Type"
    - "Location"
    - "Education Level"
    - "Major"
    - "Level"
    - "Job Function"
    - "Industry"
    For missing fields, use the value "missing".
    """
)

def extract_job_details(description):
    print("*"*10)
    prompt = PROMPT_TEMPLATE.substitute(text=description)
    response = httpx.post(
        OLLAMA_ENDPOINT,
        json={"prompt": prompt, **OLLAMA_CONFIG},
        headers={"Content-Type": "application/json"},
        timeout=240,
    )
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None

    try:
        # Parse the API response
        result = response.json()["response"].strip()
        return eval(result)  # Convert the JSON response string to a Python dictionary
    except Exception as e:
        print("Error parsing response:", e)
        return {
            "Job Title": "missing",
            "Experience": "missing",
            "Salary Range": "missing",
            "Salary Currency": "missing",
            "Job Type": "missing",
            "Location": "missing",
            "Education Level": "missing",
            "Major": "missing",
            "Level": "missing",
            "Job Function": "missing",
            "Industry": "missing",
        }
extracted_info = df["job_description"].apply(extract_job_details)

********** Done ! **********


In [45]:
extracted_df = pd.DataFrame(list(extracted_info))
df = pd.concat([df, extracted_df], axis=1)
df.to_csv("job_descriptions_with_extracted_columns1.csv", index=False)
df

,job_title,job_description,salary_string,currency,salary_lower,salary_upper,salary_avg,Predicted Category,Predicted Level,Job Title,...,Job Type,Location,Education Level,Major,Level,Job Function,Industry,Salary Range,Salary Currency,Experience In Years
0,Spa Therapist,· High school qualification with minimum 2 yea...,"QAR 4,000 - QAR 5,000 a month",QAR,4000.000000,5000.000000,4500.000000,IT,assistant,Massage Therapist/Skincare Therapist/Barber,...,Temporary,Doha,High School or Bachelor's Degree,missing,missing,Beauty and Personal Care,Beauty and Wellness,NaN,NaN,NaN
1,Registered Triage Nurse NHS (Relocation Package),Triage Nurse (Registered Nurse) – Unique Oppor...,"QAR 148,490 - QAR 178,808 a year",QAR,12374.166667,14900.666667,13637.416667,nurse,senior,Triage Nurse (Registered Nurse),...,"Full-time, Permanent",UK,Bachelor,Health,Normal,Nurse,Healthcare,NaN,NaN,NaN
2,Sales & Marketing specialist - Food & Beverage,● Minimum Experience of 3 Years in F&B Devisio...,"QAR 5,000 - QAR 6,000 a month",QAR,5000.000000,6000.000000,5500.000000,food and beverage,senior,F&B Marketing and Sales Specialist,...,"Full-time, Contract",Qatar,Bachelor's,Marketing,Mid,Sales,Food and Beverage,NaN,NaN,NaN
3,Food Service Crew (Female),LOCAL HIRE (Must be in Doha-Qatar)\nPreferably...,"QAR 2,500 a month",QAR,2500.000000,2500.000000,2500.000000,food and beverage,assistant,Food Service Crew,...,Full-time,Qatar,missing,Food and Beverage,Junior,Food Service,Hospitality or Food Industry,NaN,NaN,NaN
4,Digital Print Operator,We are hiring - Digital Print Operator\nQAR 35...,"QAR 3,400 - QAR 3,500 a month",QAR,3400.000000,3500.000000,3450.000000,IT,assistant,Digital Print Operator,...,Permanent,Qatar,missing,missing,Normal,Printing,Other,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Road Chargehand (TC23002),Emirates General Services Enterprises (_EGSE_)...,"AED3,000 a month",AED,3000.000000,3000.000000,3000.000000,HR,assistant,Chargehand,...,Full-time,U.A.E,missing,missing,missing,Construction,Other,NaN,NaN,NaN
996,Internal Auditor,Perform and control the full audit cycle inclu...,"AED7,000 - AED7,500 a month",AED,7000.000000,7500.000000,7250.000000,HR,intern,Internal Auditor,...,Full-time,missing,Bachelor,Business,Normal,Auditor,other (auditing),NaN,NaN,NaN
997,Tele sales Executive (Female),We are looking for an enthusiastic Tele sales ...,"AED2,500 - AED3,500 a month",AED,2500.000000,3500.000000,3000.000000,sales,executive,Tele sales Representative,...,Full-time,"Abu Dhabi, UAE",missing,Sales,Normal,Sales,Retail,NaN,NaN,NaN
998,Commis Chef,We are looking to hire a hard-working and effi...,"AED2,500 - AED3,500 a month",AED,2500.000000,3500.000000,3000.000000,food and beverage,executive,Commis Chef,...,Full-time,Abu Dhabi,missing,Food and Beverage,Junior,Chef,Hospitality,NaN,NaN,NaN


In [46]:
df.shape

(1000, 23)

In [47]:
df.isna().sum()

job_title                     0
job_description               0
salary_string                 0
currency                      0
salary_lower                  0
salary_upper                  0
salary_avg                    0
Predicted Category            0
Predicted Level               0
Job Title                     0
Experience                    0
Expected Salary Range        51
Expected Salary Currency     36
Job Type                      0
Location                      0
Education Level               0
Major                         0
Level                         0
Job Function                  0
Industry                      0
Salary Range                949
Salary Currency             964
Experience In Years         999
dtype: int64

In [50]:
df['Experience In Years']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
995    NaN
996    NaN
997    NaN
998    NaN
999    NaN
Name: Experience In Years, Length: 1000, dtype: object

In [44]:
extracted_info

0      {'Job Title': 'Massage Therapist/Skincare Ther...
1      {'Job Title': 'Triage Nurse (Registered Nurse)...
2      {'Job Title': 'F&B Marketing and Sales Special...
3      {'Job Title': 'Food Service Crew', 'Experience...
4      {'Job Title': 'Digital Print Operator', 'Exper...
                             ...                        
995    {'Job Title': 'Chargehand', 'Experience': '2',...
996    {'Job Title': 'Internal Auditor', 'Experience'...
997    {'Job Title': 'Tele sales Representative', 'Ex...
998    {'Job Title': 'Commis Chef', 'Experience': 'mi...
999    {'Job Title': 'Tailor', 'Experience': '3', 'Ex...
Name: job_description, Length: 1000, dtype: object

In [51]:
type(extracted_info)

pandas.core.series.Series

In [60]:
try:
    new_data = json.dumps(extracted_info[1000])
except:
    print("kk")

kk


In [59]:
new_data

'{"Job Title": "Registered Nurse / RN", "Experience": "missing", "Expected Salary Range": "AED120.00-AED157.00", "Expected Salary Currency": "AED", "Job Type": "Full-time", "Location": "USA", "Education Level": "Bachelor", "Major": "Health", "Level": "missing", "Job Function": "Nurse", "Industry": "Healthcare"}'

In [62]:
new_df=pd.read_csv('data/job_descriptions_with_extracted_columns1.csv')
new_df

,job_title,job_description,salary_string,currency,salary_lower,salary_upper,salary_avg,Predicted Category,Predicted Level,Job Title,...,Job Type,Location,Education Level,Major,Level,Job Function,Industry,Salary Range,Salary Currency,Experience In Years
0,Spa Therapist,· High school qualification with minimum 2 yea...,"QAR 4,000 - QAR 5,000 a month",QAR,4000.000000,5000.000000,4500.000000,IT,assistant,Massage Therapist/Skincare Therapist/Barber,...,Temporary,Doha,High School or Bachelor's Degree,missing,missing,Beauty and Personal Care,Beauty and Wellness,NaN,NaN,NaN
1,Registered Triage Nurse NHS (Relocation Package),Triage Nurse (Registered Nurse) – Unique Oppor...,"QAR 148,490 - QAR 178,808 a year",QAR,12374.166667,14900.666667,13637.416667,nurse,senior,Triage Nurse (Registered Nurse),...,"Full-time, Permanent",UK,Bachelor,Health,Normal,Nurse,Healthcare,NaN,NaN,NaN
2,Sales & Marketing specialist - Food & Beverage,● Minimum Experience of 3 Years in F&B Devisio...,"QAR 5,000 - QAR 6,000 a month",QAR,5000.000000,6000.000000,5500.000000,food and beverage,senior,F&B Marketing and Sales Specialist,...,"Full-time, Contract",Qatar,Bachelor's,Marketing,Mid,Sales,Food and Beverage,NaN,NaN,NaN
3,Food Service Crew (Female),LOCAL HIRE (Must be in Doha-Qatar)\nPreferably...,"QAR 2,500 a month",QAR,2500.000000,2500.000000,2500.000000,food and beverage,assistant,Food Service Crew,...,Full-time,Qatar,missing,Food and Beverage,Junior,Food Service,Hospitality or Food Industry,NaN,NaN,NaN
4,Digital Print Operator,We are hiring - Digital Print Operator\nQAR 35...,"QAR 3,400 - QAR 3,500 a month",QAR,3400.000000,3500.000000,3450.000000,IT,assistant,Digital Print Operator,...,Permanent,Qatar,missing,missing,Normal,Printing,Other,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Road Chargehand (TC23002),Emirates General Services Enterprises (_EGSE_)...,"AED3,000 a month",AED,3000.000000,3000.000000,3000.000000,HR,assistant,Chargehand,...,Full-time,U.A.E,missing,missing,missing,Construction,Other,NaN,NaN,NaN
996,Internal Auditor,Perform and control the full audit cycle inclu...,"AED7,000 - AED7,500 a month",AED,7000.000000,7500.000000,7250.000000,HR,intern,Internal Auditor,...,Full-time,missing,Bachelor,Business,Normal,Auditor,other (auditing),NaN,NaN,NaN
997,Tele sales Executive (Female),We are looking for an enthusiastic Tele sales ...,"AED2,500 - AED3,500 a month",AED,2500.000000,3500.000000,3000.000000,sales,executive,Tele sales Representative,...,Full-time,"Abu Dhabi, UAE",missing,Sales,Normal,Sales,Retail,NaN,NaN,NaN
998,Commis Chef,We are looking to hire a hard-working and effi...,"AED2,500 - AED3,500 a month",AED,2500.000000,3500.000000,3000.000000,food and beverage,executive,Commis Chef,...,Full-time,Abu Dhabi,missing,Food and Beverage,Junior,Chef,Hospitality,NaN,NaN,NaN


In [64]:
def extract_years_experience(description):
    match = re.search(r"(\d+)\s*(?:\+?\s*years?|year)", description.lower())
    return int(match.group(1)) if match else 0  

new_df["years_experience"] = new_df["job_description"].apply(extract_years_experience)

In [67]:
new_df.isna().sum()

job_title                     0
job_description               0
salary_string                 0
currency                      0
salary_lower                  0
salary_upper                  0
salary_avg                    0
Predicted Category            0
Predicted Level               0
Job Title                     0
Experience                    0
Expected Salary Range        51
Expected Salary Currency     36
Job Type                      0
Location                      0
Education Level               0
Major                         0
Level                         0
Job Function                  0
Industry                      0
Salary Range                949
Salary Currency             964
Experience In Years         999
years_experience              0
dtype: int64